In [5]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [18]:
train_df = pd.read_csv("working_data/trial_promo_training.csv")
test_df = pd.read_csv("working_data/trial_promo_testing.csv")

In [20]:
features_to_scale = ["age", "income", "avbal", "avtrans"]
min_max_scaler = preprocessing.MinMaxScaler()
train_df[features_to_scale] = min_max_scaler.fit_transform(train_df[features_to_scale])

train_df.head()

,age,income,avbal,avtrans,decision,sexNum,mstatusNum,occupationNum,educationNum,childrenNum,...,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary,children_0,children_1,children_2,children_3,children_4
0,0.481241,0.120073,0.273538,0.324016,None,0,1,6,0,2,...,0,1,0,0,0,0,0,1,0,0
1,0.035354,0.120073,0.273538,0.324016,None,1,2,1,2,0,...,0,0,0,1,0,1,0,0,0,0
2,0.490188,0.026902,0.257021,0.151600,None,0,1,0,2,0,...,0,0,0,1,0,1,0,0,0,0
3,0.111688,0.120073,0.273538,0.324016,A,1,1,1,2,0,...,0,0,0,1,0,1,0,0,0,0
4,0.135354,0.193942,0.260347,0.138515,None,1,2,2,3,0,...,0,0,0,0,1,1,0,0,0,0


In [21]:
X_train = train_df.loc[:, train_df.columns != "decision"]
y_train = train_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_train.head()

,age,income,avbal,avtrans,sexNum,mstatusNum,occupationNum,educationNum,childrenNum,sex_F,...,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary,children_0,children_1,children_2,children_3,children_4
0,0.481241,0.120073,0.273538,0.324016,0,1,6,0,2,1,...,0,1,0,0,0,0,0,1,0,0
1,0.035354,0.120073,0.273538,0.324016,1,2,1,2,0,0,...,0,0,0,1,0,1,0,0,0,0
2,0.490188,0.026902,0.257021,0.151600,0,1,0,2,0,1,...,0,0,0,1,0,1,0,0,0,0
3,0.111688,0.120073,0.273538,0.324016,1,1,1,2,0,0,...,0,0,0,1,0,1,0,0,0,0
4,0.135354,0.193942,0.260347,0.138515,1,2,2,3,0,0,...,0,0,0,0,1,1,0,0,0,0


In [22]:
y_train.head()

0    0
1    0
2    0
3    1
4    0
Name: decision, dtype: int64

In [23]:
test_df[features_to_scale] = min_max_scaler.transform(test_df[features_to_scale])
test_df.head()

X_test = test_df.loc[:, test_df.columns != "decision"]
y_test = test_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_test.head()

,age,income,avbal,avtrans,sexNum,mstatusNum,occupationNum,educationNum,childrenNum,sex_F,...,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary,children_0,children_1,children_2,children_3,children_4
0,0.704906,0.080506,0.063522,0.100964,0,1,8,2,0,1,...,1,0,0,1,0,1,0,0,0,0
1,0.139105,0.088530,0.232708,0.122881,1,2,4,2,0,0,...,0,0,0,1,0,1,0,0,0,0
2,0.514719,0.300505,0.338760,0.089771,0,0,0,3,2,1,...,0,0,0,0,1,0,0,1,0,0
3,0.148052,0.346235,0.180242,0.390105,0,2,3,3,0,1,...,0,0,0,0,1,1,0,0,0,0
4,0.425830,0.556900,0.231039,0.484708,0,0,6,1,0,1,...,0,0,1,0,0,1,0,0,0,0


### Neural network

In [24]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [25]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)


y_test_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1, 2]))
print(accuracy_score(y_test, y_test_pred))

[[207   0   0]
 [ 33   0   0]
 [ 10   0   0]]
0.828


In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=42, n_estimators=1000)
clf.fit(X_train, y_train)

y_test_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1, 2]))
print(accuracy_score(y_test, y_test_pred))

[[207   0   0]
 [ 33   0   0]
 [ 10   0   0]]
0.828
